### Description :
Using a quite simple RNN on spectrograms. Doesn't work at all.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from utils import which_set, BASE_TRAIN_FOLDER_SPEC, BASE_TRAIN_FOLDER_WAV, labels_to_ints

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, BatchNormalization, MaxPooling1D
from keras.optimizers import SGD

In [112]:
def make_sets(set_name):
    x = []
    y = []
    folder = os.path.join(BASE_TRAIN_FOLDER_SPEC, set_name)
    i = 0
    for filename in os.listdir(folder):
        if filename.startswith('.'):
            continue
        label = filename.split('.')[0]
        inputs = np.load(os.path.join(folder, filename))
        inputs = inputs.reshape(-1, inputs.shape[1] * inputs.shape[2]) # needed by RNN
        y.append([labels_to_ints[label]] * len(inputs))
        x.append(inputs)
        i += 1
    
    x = np.concatenate(x)
    y = to_categorical(np.concatenate(y), num_classes=len(labels_to_ints))
    
    return x, y

In [113]:
x_train, y_train = make_sets("train")

In [114]:
x_train.shape, y_train.shape

((51490, 2048), (51490, 31))

In [115]:
x_val, y_val = make_sets("validation")

In [116]:
x_val.shape, y_val.shape

((6798, 2048), (6798, 31))

In [117]:
x_test, y_test = make_sets("testing")

In [118]:
x_test.shape, y_test.shape

((6835, 2048), (6835, 31))

In [5]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
model.add(Dense(64, activation='relu', input_dim=2048))
model.add(Dropout(0.1))
model.add(Dense(1000, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(31, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [126]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [127]:
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val)
         )

Train on 51490 samples, validate on 6798 samples
Epoch 1/20
51490/51490 [==============================] - 4s 78us/step - loss: 3.4260 - acc: 0.0351 - val_loss: 3.4026 - val_acc: 0.0357
Epoch 2/20
51490/51490 [==============================] - 4s 70us/step - loss: 3.4129 - acc: 0.0357 - val_loss: 3.3977 - val_acc: 0.0377
Epoch 3/20
51490/51490 [==============================] - 3s 68us/step - loss: 3.4124 - acc: 0.0362 - val_loss: 3.3965 - val_acc: 0.0385
Epoch 4/20
51490/51490 [==============================] - 4s 71us/step - loss: 3.4123 - acc: 0.0341 - val_loss: 3.3961 - val_acc: 0.0362
Epoch 5/20
51490/51490 [==============================] - 4s 70us/step - loss: 3.4123 - acc: 0.0356 - val_loss: 3.3961 - val_acc: 0.0387
Epoch 6/20
51490/51490 [==============================] - 4s 70us/step - loss: 3.4122 - acc: 0.0355 - val_loss: 3.3957 - val_acc: 0.0377
Epoch 7/20
51490/51490 [==============================] - 4s 72us/step - loss: 3.4122 - acc: 0.0350 - val_loss: 3.3957 - val_acc:

In [123]:
score = model.evaluate(x_test, y_test, batch_size=128)

6835/6835 [==============================] - 0s 37us/step


In [124]:
score

[3.3946430204554727, 0.03789319678127286]